In [1]:
print("hello world")

hello world


In [2]:
# pip install crewai
# pip install 'crewai[tools]'
# pip install ollama

#### Import Required Libraries

In [3]:
from crewai import Agent, Task, Crew
import pkg_resources

In [4]:
# Get the version of CrewAI
crewai_version = pkg_resources.get_distribution("crewai").version
print(crewai_version)

0.79.4


### Define the Agents

Here, we define three agents with specific roles and goals. Each agent is responsible for a task related to image classification and description.

    Classifier Agent: Checks if the image contains an animal, uses llava:7b model to classify the animal.
    Description Agent: Describes the animal in the image. This also uses the same llava:7b model like the preceding agent.
    Information Retrieval Agent: This agent retrieves additional information or interesting facts about the animal. It uses llama2 to provide this information.


In [5]:
llm='ollama/llava-phi3:latest'

In [6]:
# 1. Image Classifier Agent (to check if the image is an animal)

classifier_agent = Agent(
    role="Image Classifier Agent",
    goal="Determine if the image is of an animal or not",
    backstory="""
    You have an eye for animals! Your job is to identify whether the input image is of an animal
    or something else.
    """,
    llm=llm# Model for image-related tasks
)


In [7]:
# 2. Animal Description Agent (to describe the animal in the image)

description_agent = Agent(
    role="Animal Description Agent {image_path}",
    goal="Describe the animal in the image",
    backstory="""
    You love nature and animals. Your task is to describe any animal based on an image.
    """,
    llm=llm# Model for image-related tasks
)


In [8]:
# 3. Information Retrieval Agent (to fetch additional info about the animal)

info_agent = Agent(
    role="Information Agent",
    goal="Give compelling information about a certain animal",
    backstory="""
    You are very good at telling interesting facts.You don't give any wrong information if you don't know it.
    """,
    llm=llm# Model for general knowledge retrieval
)

### Define Tasks for Each Agent

Each task is tied to one of the agents. Tasks describe the input, the expected output, and which agent should handle it.

    Task 1: Classify whether the image contains an animal.
    Task 2: If the image is classified as an animal, describe it.
    Task 3: Provide additional information about the animal based on the description.


In [9]:
# Task 1: Check if the image is an animal
task1 = Task(
    description="Classify the image ({image_path}) and tell me if it's an animal.",
    expected_output="If it's an animal, say 'animal'; otherwise, say 'not an animal'.",
    agent=classifier_agent
)
# Task 2: If it's an animal, describe it
task2 = Task(
    description="Describe the animal in the image.({image_path})",
    expected_output="Give a detailed description of the animal.",
    agent=description_agent
)
# Task 3: Provide more information about the animal

task3 = Task(
    description="Give additional information about the described animal.",
    expected_output="Provide at least 5 interesting facts or information about the animal.",
    agent=info_agent
)



#### Managing Agents and Tasks with a Crew

A Crew is set up to manage the agents and tasks. It coordinates the tasks sequentially and provides the results based on the agents’ chains of thought.

In [10]:
# Crew to manage the agents and tasks

crew = Crew(
agents=[classifier_agent, description_agent, info_agent],
tasks=[task1, task2, task3],
verbose=True
)



In [11]:
# Execute the tasks with the provided image path
result = crew.kickoff(inputs={'image_path': 'racoon.jpg'})

# Agent: Image Classifier Agent
## Task: Classify the image (racoon.jpg) and tell me if it's an animal.


# Agent: Image Classifier Agent
## Final Answer: 
animal


# Agent: Animal Description Agent racoon.jpg
## Task: Describe the animal in the image.(racoon.jpg)


# Agent: Animal Description Agent racoon.jpg
## Final Answer: 
The image features a raccoon in mid-stride. The raccoon is medium-sized, with a rich brown coat that has distinctive black stripes and patches on its face, ears, back legs, tail, and hind foot. Its front paws are held up as it walks across the ground, showcasing its agility and balance. Raccoons have dextrous hands, with five fingers each and a strong grip that allows them to climb trees and open containers. In this image, you can see these features clearly, along with their inquisitive nature as they explore their surroundings.


# Agent: Information Agent
## Task: Give additional information about the described animal.


# Agent: Information Agent
## Final Ans